<a href="https://colab.research.google.com/github/patryktk/175IC-machine-learning-21176/blob/main/Zad10/Zad10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

import pandas as pd
import numpy as np

In [ ]:
def to_category(df_raw):
    for key in df_raw.keys():
        if df_raw[key].dtype == "object":
            df_raw[key] = df_raw[key].astype("category")
    return df_raw

def data_target_split(df):
    data = df.copy()
    target = data.pop('charges')

    return data, target

def scale(df, min_max_scaler):  
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(df)
    min_max_scaler
    df = pd.DataFrame(x_scaled)

    return df


def custom_scale(df, normalize_tab=normalize_data_tab, target=False):
    if target == False:
        for i, key in enumerate(['age', 'bmi', 'children']):
            df[key] = (df[key] - normalize_tab[i*2+1]) / (normalize_tab[i*2] - normalize_tab[i*2+1])

    
    else:
        df = (df - normalize_tab[1]) / (normalize_tab[0] - normalize_tab[1])
    
    return df

def normalized_to_real(x, normalize_tab=normalize_target_tab):
    return x* (normalize_tab[0]-normalize_tab[1]) + normalize_tab[1]

df_raw = pd.read_csv("https://storage.googleapis.com/esmartdata-courses-files/ml-course/insurance.csv")


print(df_raw[:10])


df = df_raw.copy()

df = to_category(df)

df = pd.get_dummies(df, drop_first=True)

data, target = data_target_split(df)

normalize_data_tab = np.array([data['age'].max(), data['age'].min(), 
                               data['bmi'].max(), data['bmi'].min(), 
                               data['children'].max(), data['children'].min()])
normalize_target_tab = np.array([target.max(), target.min()])

data = custom_scale(data)
target = custom_scale(target, normalize_target_tab, True)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")


model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(8,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='msle')
hist = model.fit(X_train, y_train, batch_size=10, epochs=50, validation_data=(X_test, y_test))